In [ ]:
# Data Exploration for Protein Function Classifier
# ================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os
import sys

# Add parent directory to path so we can import our modules
sys.path.append('..')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("Imports complete!")

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw/uniprot_enzymes.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Basic dataset info
print("Dataset overview:")

print(f"\nTotal sequences: {len(df)}")
print(f"Unique organisms: {df['organism'].nunique()}")
print(f"Missing values:\n{df.isnull().sum()}")

print("\nData types:")
print(df.dtypes)

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Get class counts
class_data = df.groupby(['ec_class', 'ec_name']).size().reset_index(name='count')
class_data = class_data.sort_values('ec_class')

# Bar plot
ax1 = axes[0]
colors = sns.color_palette("husl", 7)
bars = ax1.bar(class_data['ec_class'], class_data['count'], color=colors)
ax1.set_xlabel('EC Class', fontsize=12)
ax1.set_ylabel('Number of Sequences', fontsize=12)
ax1.set_title('Distribution of Enzyme Classes', fontsize=14)
ax1.set_xticks(range(1, 8))

# Add count labels on bars
for bar, count in zip(bars, class_data['count']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             str(count), ha='center', va='bottom', fontsize=10)

# Pie chart
ax2 = axes[1]
labels = [f"EC {row['ec_class']}\n{row['ec_name']}" for _, row in class_data.iterrows()]
ax2.pie(class_data['count'], labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Proportion of Each EC Class', fontsize=14)

plt.tight_layout()
plt.savefig('../figures/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved figure to figures/class_distribution.png")

In [ ]:
# Sequence length distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram by class
ax1 = axes[0]
for ec_class in sorted(df['ec_class'].unique()):
    subset = df[df['ec_class'] == ec_class]['length']
    ax1.hist(subset, bins=50, alpha=0.5, label=f'EC {ec_class}', density=True)

ax1.set_xlabel('Sequence Length (amino acids)', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Sequence Length Distribution by EC Class', fontsize=14)
ax1.legend()

# Box plot
ax2 = axes[1]
df.boxplot(column='length', by='ec_class', ax=ax2)
ax2.set_xlabel('EC Class', fontsize=12)
ax2.set_ylabel('Sequence Length', fontsize=12)
ax2.set_title('Sequence Length by EC Class', fontsize=14)
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.savefig('../figures/sequence_length_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# Print statistics
print("\nSequence Length Statistics by EC Class:")
print("-" * 50)
length_stats = df.groupby('ec_class')['length'].agg(['count', 'mean', 'std', 'min', 'max'])
length_stats = length_stats.round(1)
print(length_stats)

In [ ]:
# Calculate amino acid composition for each sequence
def calculate_aa_composition(sequence):
    """Calculate frequency of each amino acid."""
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    length = len(sequence)
    if length == 0:
        return {aa: 0 for aa in amino_acids}
    return {aa: sequence.count(aa) / length for aa in amino_acids}

compositions = df['sequence'].apply(calculate_aa_composition)
composition_df = pd.DataFrame(compositions.tolist())
composition_df['ec_class'] = df['ec_class'].values

print(f" Calculated composition for {len(composition_df)} sequences")

# Average composition by class
avg_composition = composition_df.groupby('ec_class').mean()
print("\nAverage amino acid composition by EC class:")
avg_composition.head()

In [ ]:
# Heatmap of amino acid composition
plt.figure(figsize=(14, 6))

# Create heatmap
sns.heatmap(avg_composition, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=list('ACDEFGHIKLMNPQRSTVWY'),
            cbar_kws={'label': 'Frequency'})

plt.xlabel('Amino Acid', fontsize=12)
plt.ylabel('EC Class', fontsize=12)
plt.title('Average Amino Acid Composition by Enzyme Class', fontsize=14)
plt.tight_layout()
plt.savefig('../figures/aa_composition_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

# Find most discriminative amino acids (highest variance between classes)
aa_variance = avg_composition.var().sort_values(ascending=False)
print("\nMost Discriminative Amino Acids (highest variance between classes):")
print("-" * 50)
for aa, var in aa_variance.head(5).items():
    print(f"  {aa}: variance = {var:.6f}")

In [ ]:
# Top source organisms
plt.figure(figsize=(12, 6))

top_organisms = df['organism'].value_counts().head(15)
top_organisms.plot(kind='barh', color=sns.color_palette("viridis", 15))

plt.xlabel('Number of Sequences', fontsize=12)
plt.ylabel('Organism', fontsize=12)
plt.title('Top 15 Source Organisms in Dataset', fontsize=14)
plt.tight_layout()
plt.savefig('../figures/top_organisms.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTotal unique organisms: {df['organism'].nunique()}")

In [ ]:
# Save processed data for next steps
# Save cleaned dataset
df.to_csv('../data/processed/cleaned_enzymes.csv', index=False)
print(f" Saved cleaned_enzymes.csv ({len(df)} sequences)")

# Save amino acid composition features
composition_df.to_csv('../data/processed/aa_composition.csv', index=False)
print(f" Saved aa_composition.csv ({len(composition_df)} samples, {len(composition_df.columns)} features)")
print("""
Key Findings:
1. Dataset has {total} sequences across 7 EC classes
2. Classes are well-balanced (~700-780 each)
3. Sequence lengths range from {min_len} to {max_len} amino acids
4. Most sequences come from {top_org}

""".format(
    total=len(df),
    min_len=df['length'].min(),
    max_len=df['length'].max(),
    top_org=df['organism'].value_counts().index[0]
))